In [86]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [93]:
from IPython.display import clear_output, Image, display
import PIL.Image
import io
import json
import torch
import numpy as np
from _lxmert.processing_image import Preprocess
from _lxmert.visualizing_image import SingleImageViz
from _lxmert.modeling_frcnn import GeneralizedRCNN
from _lxmert._utils import Config
import _lxmert._utils
from transformers import LxmertForQuestionAnswering, LxmertTokenizer
import wget
import pickle
import os
from transformers import LxmertForPreTraining
# for visualizing output
def showarray(a, fmt="jpeg"):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
lxmert_base = LxmertForPreTraining.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)
# load models and model components
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg)

image_preprocess = Preprocess(frcnn_cfg)

lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /afs/cs.pitt.edu/usr0/arr159/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [44]:
import _lxmert
OBJ_URL = "https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/genome/1600-400-20/objects_vocab.txt"
objids = _lxmert._utils.get_data(OBJ_URL)


In [96]:
import torch.nn as nn
import random

def get_object_detection_output(img_paths):
    images, sizes, scales_yx = image_preprocess(img_paths)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    return output_dict
def get_objects(output_dict):
    objects=[]
    for tmp in zip(output_dict.get("obj_ids"),output_dict.get("obj_probs")):
        objects.append([objids[i] for i, p in zip(tmp[0].tolist(), tmp[1].tolist()) if p > 0.5])
    return objects
def pretrained_model_fwd_pass(img_paths, txt):
    output_dict=get_object_detection_output(img_paths)
    objects=get_objects(output_dict)
    normalized_boxes = output_dict.get("normalized_boxes").to(device)
    features = output_dict.get("roi_features").to(device)
    inputs = lxmert_tokenizer(
        txt,
        padding="max_length",
        max_length=77,
        truncation=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    ).to(device)
    cross_relationship_score= nn.Sigmoid()(lxmert_base(input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        visual_feats=features,
        visual_pos=normalized_boxes,
        token_type_ids=inputs.token_type_ids,
        output_attentions=False)['cross_relationship_score'])
    shuffled_ids=list(range(len(txt)))
    random.shuffle(shuffled_ids)
    inputs = lxmert_tokenizer(
        [txt[_id] for _id in shuffled_ids],
        padding="max_length",
        max_length=77,
        truncation=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    ).to(device)
    actually_different=[shuffled_ids[_id] == _id for _id in list(range(len(txt)))]
    random_cross_relationship_score= nn.Sigmoid()(lxmert_base(input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        visual_feats=features,
        visual_pos=normalized_boxes,
        token_type_ids=inputs.token_type_ids,
        output_attentions=False)['cross_relationship_score'])
    return objects, cross_relationship_score.tolist(), random_cross_relationship_score.tolist(), actually_different

False

In [4]:
import importlib 
import Dataset
importlib.reload(Dataset)
from Dataset import ReduceMAMIDataset, collate2
num_epochs = 30

# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.485, 0.456, 0.406),
#                           (0.229, 0.224, 0.225))])
from params import *
train_dataset = ReduceMAMIDataset(MAX_LEN, MAX_VOCAB, split='train', path_to_dataset='./Data/MASKED_TEXT_TRAINING', transform=None)
val_dataset = ReduceMAMIDataset(MAX_LEN, MAX_VOCAB, split='val', path_to_dataset='./Data/MASKED_TEXT_TRAINING', transform=None)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, num_workers=8, collate_fn=collate2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate2)

dataloader = {
    'train': train_loader,
    'val': val_loader
}

num_classes = 2



[nltk_data] Downloading package stopwords to
[nltk_data]     /afs/cs.pitt.edu/usr0/arr159/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /afs/cs.pitt.edu/usr0/arr159/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
train_dataset = ReduceIronicMEMEDataset(MAX_LEN, MAX_VOCAB, path_to_dataset_folder='./Data/Misogynistic-MEME_pt1', path_to_data={'images':'Images', 'csv':'table.csv'}, transform=None)


In [8]:
import pandas as pd
import os
path_to_dataset_folder='./Data/Misogynistic-MEME_pt1'
path_to_data={'images':'Images', 'csv':'table.csv'}
df = data = pd.read_csv(os.path.join(path_to_dataset_folder, path_to_data['csv']),sep=";")

In [14]:
df[df['misogynisticDE']==1][df['ironicDE']==1]

/tmp/ipykernel_23714/398518272.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['misogynisticDE']==1][df['ironicDE']==1]


,memeID,text,misogynisticDE,aggressiveDE,ironicDE,misogynisticCS,aggressiveCS,ironicCS,confidence_M_CS,confidence_A_CS,confidence_I_CS
0,ENG01,The way every man feels when a woman is driving,1,0,1,1,0,1,0.6716,0.6716,0.6716
1,ENG02,Women yes it's a woman,1,0,1,1,0,1,1.0000,1.0000,1.0
2,ENG03,"I dont' understand, this car has 3 pedals and ...",1,0,1,1,0,1,1.0000,1.0000,1.0
3,ENG04,They said I didn't belong in the garage so I b...,1,1,1,1,0,1,1.0000,0.6672,0.6672
4,ENG05,Women drivers it's the only possible explanation,1,0,1,1,0,1,1.0000,0.6664,1.0
...,...,...,...,...,...,...,...,...,...,...,...
389,ENG390,The floor is women rights,1,1,1,1,0,1,1.0000,0.6623,0.6841
390,ENG391,"Lol basically. Women, men.",1,0,1,1,0,1,0.6820,0.6820,0.682
391,ENG392,Stupid slow driver.. o its a woman,1,1,1,1,0,1,1.0000,1.0000,0.6587
392,ENG393,what men play with vs what women play with,1,0,1,0,0,0,0.6768,0.0000,0.0


In [98]:
toy_data=[train_dataset[0], train_dataset[1]]
batch_img_id, batch_img_paths, batch_text=collate2(toy_data)
pretrained_model_fwd_pass(batch_img_paths, batch_text)

([['nose',
   'ear',
   'face',
   'dog',
   'eye',
   'eye',
   'eye',
   'eye',
   'mouth',
   'hair',
   'paw',
   'paw',
   'mouth',
   'hand'],
  ['eye',
   'eye',
   'face',
   'eye',
   'face',
   'face',
   'mouth',
   'nose',
   'dog',
   'face',
   'mouth',
   'dog',
   'mouth']],
 [[0.861376166343689, 0.16290029883384705],
  [0.5193160772323608, 0.4576598107814789]],
 [[0.861376166343689, 0.16290029883384705],
  [0.5193160772323608, 0.4576598107814789]],
 [True, True])

In [75]:
inputs = lxmert_tokenizer(
        shuffle(batch_text),
        padding="max_length",
        max_length=77,
        truncation=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )

NameError: name 'shuffle' is not defined

In [74]:
inputs

{'input_ids': tensor([[  101,  1045,  5086,  1037, 17074,  2138,  1045,  2001,  2894,  2006,
          2026,  5798, 22843,  2014,  2005,  2321,  2781,  2059,  2081,  2014,
          2079,  1996, 10447,  1998, 11641,  2005,  1996,  2717,  1997,  1996,
          3178,  2033,  4168,  3993,  1012,  4012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  7966,  2056,  2026,  8407,  2298,  2066,  2070,  6167,
          4842,  2725,  1037,  3975,  1999,  2070, 17074,  6879,  1998,  2085,
          1045,  2064,  1005,  1056,  4895, 19763,  2009,  1012,  1030, 14405,
          9299,  2015,  2005,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0, 

In [8]:
import pickle
coco_data_path='/own_files/datasets/mscoco/train2014/'
with open(f'/afs/cs.pitt.edu/usr0/arr159/erhan_code/t/t/data/COCO2017_train_capdata.pkl', 'rb') as f:
    coco_df=pickle.load(f)

In [24]:
import random
caption, _, _, _= random.choice(coco_df['capdata'][0])
caption


'a long table with a flower arrangement in the middle for meetings'

In [17]:
from torch.utils.data import Dataset
import random
import os
import pickle
class COCODataset(Dataset):
    def __init__(self, path='/own_files/datasets/mscoco'):
        with open(f'/afs/cs.pitt.edu/usr0/arr159/erhan_code/t/t/data/COCO2017_train_capdata.pkl', 'rb') as f:
            df=pickle.load(f)
            
        self.ids=df['path']
        self.captions=df['capdata']
        self.img_paths=[self.get_path(path, img_pth)  for img_pth in df['path']]
    def __len__(self):
        return len(self.ids)
    def get_path(self, path, filename):
        return os.path.join(path, ('train2014' if 'train2014' in filename else 'val2014'), filename)
    def __getitem__(self, i):
        caption, _, _, _= random.choice(self.captions[i])# extract cap
        return (self.ids[i], caption, self.img_paths[i])
    @staticmethod
    def collate_fn(batch):
        ids=[data[0] for data in batch]
        img_paths=[data[2] for data in batch]
        captions=[data[1] for data in batch]
        return ids, img_paths, captions


In [20]:
import pickle
with open('coco_captions.pickle', 'wb') as f:
    captions=COCODataset().captions
    captions=[random.choice(i)[0] for i in captions]# extract cap
    
    pickle.dump(captions, f)

In [52]:
lxmert_processed_files=["masked_df_ironic_meme_lxmert.csv","coco_lxmert.csv", "non_masked_df_ironic_meme_lxmert.csv", "masked_df_mami_lxmert2.csv", "not_masked_df_mami_lxmert2.csv"]

In [53]:
# according to this github issue, using the second logit as a match - https://github.com/huggingface/transformers/issues/7266#issuecomment-748211135 
import re
def fix_dataframe(path_to_df):
    def process_str_arr(x):
        return [float(i) for i in re.split('\[|\]|,',x) if i != '']

    df=pd.read_csv(path_to_df)
    
    df['sim_match']=df['sim'].apply(lambda x: process_str_arr(x)[1])
    df['sim_mismatch']=df['sim'].apply(lambda x: process_str_arr(x)[0])
    df['random_sim_match']=df['random_sims'].apply(lambda x: process_str_arr(x)[1])
    df['random_sim_mismatch']=df['random_sims'].apply(lambda x: process_str_arr(x)[0])

    return df

In [54]:
for file_name in lxmert_processed_files:
    print(file_name)
    df=fix_dataframe(file_name)
    print(round(df['sim_match'].mean(), 4), round(df['sim_match'].std(), 4))
    print(round(df['random_sim_match'].mean(), 4), round(df['random_sim_match'].std(), 4))
    


masked_df_ironic_meme_lxmert.csv
0.2521 0.1392
0.2243 0.124
coco_lxmert.csv
0.4639 0.3108
0.0242 0.0625
non_masked_df_ironic_meme_lxmert.csv
0.2881 0.1512
0.2594 0.1339
masked_df_mami_lxmert2.csv
0.2673 0.1274
0.247 0.1139
not_masked_df_mami_lxmert2.csv
0.323 0.1463
0.3106 0.1463


In [56]:
df=fix_dataframe('masked_df_ironic_meme_lxmert.csv')
meta_data_df=pd.read_csv('Data/MASKED_IRONIC_MEME_TEXT_TRAINING/table.csv', sep=';')


In [67]:
meta_data_df.columns

Index(['memeID', 'text', 'misogynisticDE', 'aggressiveDE', 'ironicDE',
       'misogynisticCS', 'aggressiveCS', 'ironicCS', 'confidence_M_CS',
       'confidence_A_CS', 'confidence_I_CS'],
      dtype='object')

In [78]:
meta_data_df#['ironicDE']
df['memeID']=df['img_ids'].apply(lambda x: re.split('res_|.jpg',x)[-2])

merged_df=meta_data_df.merge(df, on='memeID')


In [79]:
def get_scores_by_attr(attribute):
    condition=merged_df[attribute]==1
    print('With', attribute, ': sim:', round(merged_df[condition]['sim_match'].mean(),4), round(merged_df[condition]['sim_match'].std(),4), 'random sim:', round(merged_df[condition]['random_sim_match'].mean(),4), round(merged_df[condition]['random_sim_match'].std(), 4))
    condition=merged_df[attribute]==0
    print('Without', attribute, ': sim:', round(merged_df[condition]['sim_match'].mean(), 4), round(merged_df[condition]['sim_match'].std(), 4), 'random sim:', round(merged_df[condition]['random_sim_match'].mean(), 4), round(merged_df[condition]['random_sim_match'].std(), 4))

In [81]:
print("mean and std")
get_scores_by_attr('misogynisticDE')
get_scores_by_attr('ironicDE')

mean and std
With misogynisticDE : sim: 0.2543 0.155 random sim: 0.2376 0.1328
Without misogynisticDE : sim: 0.2499 0.1216 random sim: 0.211 0.1132
With ironicDE : sim: 0.2584 0.1594 random sim: 0.2334 0.1274
Without ironicDE : sim: 0.2507 0.1345 random sim: 0.2223 0.1233


In [74]:
import re
test_str='[0.993421733379364, 0.006969502195715904]'
[float(i) for i in re.split('\[|\]|,',test_str) if i != '']
#[str_num.split('[]')[0] for str_num in .split()]
test_str='res_ENGN339.jpg'
re.split('res_|.jpg',test_str)[-2]

'ENGN339'

In [51]:
df=fix_dataframe(file_name)
print(df['img_ids'])
print(round(df['sim_match'].mean(), 4), round(df['sim_match'].std(), 4))
print(round(df['random_sim_match'].mean(), 4), round(df['random_sim_match'].std(), 4))


Index(['Unnamed: 0', 'img_ids', 'sim', 'random_sims', 'img_random_sims',
       'objects', 'sim_match', 'sim_mismatch', 'random_sim_match',
       'random_sim_mismatch'],
      dtype='object')
0.323 0.1463
0.3106 0.1463
